In [1]:
from queue import Queue
import re

Далее представлена функция построения регулярного выражения из графа-источника make_regex. 

На вход получаем описание графа источника (имена вершин, список смежности, список финальных вершин)
На выходе получаем массив заполненный результатами для различных стоков и для
получения итогового регулярного выражения достаточно объединить их знаком ’+’.

Алгоритм представляет из себя 2 обхода в ширину. На первом проходе ищем циклы и наколение путей в терминальных вершинах. На втором проходе формируем финальные регулярные выражения.

In [2]:
def make_regex(s, d, neighbours, names):
    Q = Queue()
    Q.put(s)
    d_ = [''] * len(neighbours)
    d_[s] = names[s]
    result = []
    while not Q.empty():
        current_v = Q.get()
        for u in neighbours[current_v]:
            if u in d:
                result.append(d_[current_v] + ' ' + names[u])
            elif d_[u] == '':
                d_[u] = d_[current_v] + ' ' + names[u]
                Q.put(u)
            else:
                d_[u] = '(' + d_[current_v] + ')*(' + names[u]
    
    Q.put(s)
    while not Q.empty():
        current_v = Q.get()
        for u in neighbours[current_v]:
            if u == s:
                continue
            if u in d:
                result.append(d_[current_v] + ' ' + names[u] + ')')
            else:
                d_[u] = d_[current_v] + ' ' + names[u]
                Q.put(u)
    return result

Данная функция объединяет результат работы make_regex в одно регулярное выражение.

In [3]:
def uniteResult(ls):
    it = iter(ls)
    out = next(it)
    for inst in it:
        out += ' + ' + inst

    return out

Далее представлены функции для обновления графа источника.

Проверяется для каждой из команд: существует ли 2 различных состояния (или одно
состояние, но 2 различных события на переходах), из которых вызывается эта команда. Если это условие выполнено, то команду нужно ’размножить’ по числу таких состояний. В описаниях переходов заменяем вызовы команд на введённые.

In [6]:
def updateTransition(sm_ls, state1, state2, command):
    for state_machine in sm_ls:
        for transition in state_machine['transition']:
            if transition['actions'][1] == command:
                if transition['states'][0] == state1:
                    transition['actions'][1] = command + "_" + state1
                if transition['states'][0] == state2:
                    transition['actions'][1] = command + "_" + state2

In [7]:
def splitVertices(sm_ls):
    for state_machine1 in sm_ls:
        for transition1 in state_machine1['transition']:
            for state_machine2 in sm_ls:
                for transition2 in state_machine2['transition']:
                    command = transition1['actions'][1]
                    if command == transition2['actions'][1] \
                    and transition1['states'][0] != transition2['states'][0] \
                    and command != '':
                        if command in state_machine1['interfaces']: state_machine1['interfaces'].remove(command)
                        if command in state_machine2['interfaces']: state_machine2['interfaces'].remove(command)
                        
                        state1 = transition1['states'][0]
                        state2 = transition2['states'][0]
                        
                        new_command1 = command + "_" + state1
                        new_command2 = command + "_" + state2
                        
                        updateTransition(sm_ls, state1, state2, command)
                        
                        transition1['actions'][1] = new_command1
                        transition2['actions'][1] = new_command2
                        
                        state_machine1['interfaces'].append(new_command1)
                        state_machine2['interfaces'].append(new_command2)
                        
                        if command in state_machine1['inner']:
                            state_machine1['inner'].remove(command)
                            state_machine1['inner'].append(new_command1)
                        if command in state_machine2['inner']:
                            state_machine2['inner'].remove(command)
                            state_machine2['inner'].append(new_command2)

В следующих функциях осуществляется поиск терминальных вершин.

In [8]:
def findFinallState(state_machine):
    for state in state_machine['transition']:
        if state['states'][1] == 'exit': return state['states'][0]

def findFinallActions(finall_state, state_machine, vertices_names):
    out = []
    for state in state_machine['transition']:
        if state['states'][1] == finall_state:
            if state['actions'][1] == '':
                out.append(vertices_names.index(state['actions'][0]))
            else:
                out.append(vertices_names.index(state['actions'][1]))
            del state

    return out

Функция getGraph строит граф согласно описанию.

Реализует следующие пункты:
* Выделить вершины
* Разможить команды если необходимо
* Добавить дуги

In [13]:
def getGraph(sm_ls):
    vertices_names = []
    finall = []
    for state_machine in sm_ls:
        vertices_names.extend(state_machine['interfaces'])
        finall_state = findFinallState(state_machine)
        finall.extend(findFinallActions(finall_state, state_machine, vertices_names))

    neighbours = [[] for i in range(len(vertices_names))]

    for state_machine in sm_ls:
        for state in state_machine['transition']:
            if state['actions'][1] != '' and state['actions'][0] != '':
                v_idx = vertices_names.index(state['actions'][0])
                value = vertices_names.index(state['actions'][1])
                if not value in neighbours[v_idx]: neighbours[v_idx].append(value)

    for state_machine in sm_ls:
        for state in state_machine['transition']:
            action = state['actions'][1]
            state_buffer = state['states'][1]
            if action in state_machine['inner']:
                for state_in in state_machine['transition']:
                    if state_buffer == state_in['states'][0] \
                     and state_in['actions'][0] != '':
                        v_idx = vertices_names.index(action)
                        value = vertices_names.index(state_in['actions'][0])
                        if not value in neighbours[v_idx]: neighbours[v_idx].append(value)
                            
    for state_machine in sm_ls:
        for state in state_machine['transition']:
            if state['actions'][1] == '':
                action = state['actions'][0]
                state_buffer = state['states'][1]
                if action in state_machine['inner']:
                    for state_in in state_machine['transition']:
                        if state_buffer == state_in['states'][0] \
                         and state_in['actions'][0] != '':
                            v_idx = vertices_names.index(action)
                            value = vertices_names.index(state_in['actions'][0])
                            if not value in neighbours[v_idx]: neighbours[v_idx].append(value)

    return vertices_names, neighbours, finall

Далее находятся вспомогательные функции

In [5]:
def processFile(data):
    data = data.casefold()
    data = re.sub(r"end(\n)+", r"end", data)
    row_sm_list = [i for i in data.casefold().split('end') if i != '']
    
    sm_list = []
    for row_sm in row_sm_list:
        sm = getStateMachine(row_sm)
        if not sm is None: sm_list.append(sm)
    
    return sm_list

In [4]:
def getStateMachine(data):
    commands = [i for i in data.split('\n') if i != '']
    it = iter(commands)
    
    state_machine = dict()
    fields = iter(['provided', 'inner', 'state'])
    
    next_field = next(fields)

    try:
        state_machine["name"] = next(it)
        if next(it) == 'provided':
            next_field = next(fields)
            state_machine['interfaces'] = []

            next_line = next(it)
            while next_line != 'inner':
                state_machine['interfaces'].append(next_line)
                next_line = next(it)
        else:
            print("Invalid sintax, \"provided\" missing")
            return None
            
        next_field = next(fields)
        state_machine['inner'] = []

        next_line = next(it)
        while next_line != 'state':
            state_machine['inner'].append(next_line)
            state_machine['interfaces'].append(next_line)
            next_line = next(it)

        state_machine['transition'] = []

        for next_line in it:
            buffer = next_line.split('->')
            state_machine['transition'].append({'states': [buffer[0], buffer[-1]],
                                'actions': [i for i in buffer[1].split("/")]})
        
        return state_machine
    
    except StopIteration:
        print("Invalid sintax, \"{}\" missing".format(next_field))
        return None

Демонстрация работы алгоритма. Построенные граф и соответсвующее регулярное выражение.

Пример №1

In [14]:
with open('test1.txt', 'r', encoding='utf-8') as f:
    sm_ls = processFile(f.read())
    splitVertices(sm_ls)
    vertices_names, neighbours, finall = getGraph(sm_ls)
    print("names: ", vertices_names)
    print("neighbours: ", neighbours)
    print("finall: ", finall)
    print()
    ans = uniteResult(make_regex(vertices_names.index("нужда"), finall, neighbours, vertices_names))
    print(ans)

names:  ['заказ', 'готово', 'делать', 'отказ_готов', 'отказ_занят', 'поставка', 'нужда', 'использовать', 'возврат_готов', 'возврат_занят']
neighbours:  [[2, 3], [4, 5], [1], [], [], [7, 9], [0, 8], [6], [], []]
finall:  [3, 4, 8, 9]

нужда возврат_готов + нужда заказ отказ_готов + нужда заказ делать готово отказ_занят + нужда заказ делать готово поставка возврат_занят + (нужда заказ делать готово поставка использовать)*(нужда возврат_готов) + (нужда заказ делать готово поставка использовать)*(нужда заказ отказ_готов) + (нужда заказ делать готово поставка использовать)*(нужда заказ делать готово отказ_занят) + (нужда заказ делать готово поставка использовать)*(нужда заказ делать готово поставка возврат_занят)


Пример №2

In [15]:
with open('test2.txt', 'r', encoding='utf-8') as f:
    sm_ls = processFile(f.read())
    splitVertices(sm_ls)
    vertices_names, neighbours, finall = getGraph(sm_ls)
    print("names: ", vertices_names)
    print("neighbours: ", neighbours)
    print("finall: ", finall)
    print()
    ans = uniteResult(make_regex(vertices_names.index("заказ"), finall, neighbours, vertices_names))
    print(ans)

names:  ['заказ', 'возврат', 'готово', 'отказ', 'изготовить']
neighbours:  [[4], [], [0, 1], [], [2, 3]]
finall:  [1, 3]

заказ изготовить отказ + заказ изготовить готово возврат + (заказ изготовить готово)*(заказ изготовить отказ) + (заказ изготовить готово)*(заказ изготовить готово возврат)
